#### Memory Management using Python

1. Memory management in Python involves the combination of automatic garbage collection, reference counting and various internal optimization techniques to efficiently manage memory allocation and de-allocation


#### Reference Counting:

1. Reference counting is the primary method python uses to manage the memory

2. Every object in Python uses reference counting to indicate the count of references pointing to it

3. If the reference count falls to zero, then the memory allocated to that object will be de-allocated

In [23]:
## Import sys library for reference count
import sys

## create an object
a=[]
print(f'Get Reference for a is {sys.getrefcount(a)}')
## This code will give the output as referenceCount of 2
## Because, object creation is refering itself once, and then using 'getrefcount(a)' function will refer the object again
## So, the total object references here are 2

Get Reference for a is 2


In [24]:
b=a
print(f'Reference count of a is {sys.getrefcount(a)}')
print(f'Reference count of b is {sys.getrefcount(b)}')
## These will give 3 because, object refered by
## a
## b
## getrefcount(b)

Reference count of a is 3
Reference count of b is 3


In [25]:
b = []
print(f'Reference count of a is {sys.getrefcount(a)}')
print(f'Reference count of b is {sys.getrefcount(b)}')
## Again this will give 2 as 'a' has been refered twice since we re-assigned 'b' with another value

Reference count of a is 2
Reference count of b is 2


In [26]:
del a
del b
## Here, the reference count will become 0 for both the objects. So, the memory will be de-allocated
print(f'Reference count of a is {sys.getrefcount(a)}')
print(f'Reference count of b is {sys.getrefcount(b)}')

NameError: name 'a' is not defined

#### Advantage of Reference Counting:

1. CPython uses reference counting as the primary memory management strategy.

#### Disadvantage in Reference Counting:

1. Whenever a reference cycle is formed, that memory can't be de-allocated using Reference Couting. So, to overcome this, we use a cyclic garbage collector

2. A separate cycle detector handles objects involved in reference cycles (e.g., a refers to b and b refers to a), where refcounts never drop to 0 by themselves.


#### Garbage Collection

1. Python uses separate cyclic garbage collector to handle reference cycles. i.e., a=b, b=a where referencing never comes to zero automatically


In [ ]:
## Easiest example

import gc

## Create a list
x = []                                  ## external reference to the list
print(x)

## Append the list to itself
x.append(x)                             ## internal reference of the list to the list itself
print(x)

## reference count
print(f'Reference Count of X is {sys.getrefcount(x)}')

## delete the external reference
del x                                   ## memory won't be de-allocated as the internal reference still exist and we can't reach the internal reference

## without reference count coming to zero, python won't de-allocate the memory automatically

## to de-allocate the memory
gc.collect()                            ## Garbage collector will break the reference cycles

## gc.collect() will return number of objects are unreachable and breaks those cycle. Be it 1 or 1000 that were formed in earlier codes as well

[]
[[...]]
Reference Count of X is 3


1

#### What’s happening:
​

The list object has an element that points back to the same list.

Before del x, there are:

1 external reference (x variable),

1 internal reference (from the list to itself).

After del x, the external reference is gone, but the internal self‑reference keeps the reference count > 0, so normal ref counting would never free it.

The list is now unreachable from your code but still self‑referencing → classic reference cycle.

Python’s cyclic garbage collector periodically finds such unreachable cycles and frees them.

In [ ]:
import gc
## To get per generation statistics
print(gc.get_stats())

## In this three dictionaries are returned because Python’s garbage collector is generational and uses three generations (0, 1, 2)
'''
Python's GC groups objects into three generations based on how long they’ve lived: 

Generation 0 (index 0 in the list):

    Newly created, “young” objects.

    Collected most frequently.

Generation 1 (index 1):

    Objects that survived at least one collection in gen 0.

    Collected less often than gen 0.

Generation 2 (index 2):

    Long‑lived objects that survived multiple collections.

    Collected least frequently.
'''

[{'collections': 260, 'collected': 1178, 'uncollectable': 0}, {'collections': 23, 'collected': 623, 'uncollectable': 0}, {'collections': 2, 'collected': 74, 'uncollectable': 0}]


### gc.get_stats()

Gives the per-generation statistics about Python's garbage collector since interpreter started

Each dictionary currently has 3 keys:

collections: How many times this generation has been collected

collected: How many objects were freed from this generation

uncollected: How many objects were found but could not be collected



    So in the example above:

    Generation 0 has been collected 260 times and freed 1178 objects.

    Generation 2 has been collected 2 times and freed 74 objects, etc.

When this is useful

gc.get_stats() is mainly for monitoring and tuning memory/GC behavior:

1. Check how often each generation is collected.

2. See if many objects are marked uncollectable (could hint at leaks due to reference cycles).
​


In [ ]:
## Example code to clearly understand the gc.get_stats() operation

import gc

gc.collect()                                ## to reset the collector
print(f'Before: ', gc.get_stats())

## create and drop some temporary objects
for _ in range(100000):
    x= [i for i in range (10000)]
    ## objects here are freed frequently as x has been re-assigned for each iteration

gc.collect()
print(f'After: ', gc.get_stats())

## Since the loop we used here is a tiny one, it is not pushing gen 0/1 over their thresholds so only gen 2 is changing

Before:  [{'collections': 274, 'collected': 1445, 'uncollectable': 0}, {'collections': 24, 'collected': 909, 'uncollectable': 0}, {'collections': 16, 'collected': 937, 'uncollectable': 0}]
After:  [{'collections': 274, 'collected': 1445, 'uncollectable': 0}, {'collections': 24, 'collected': 909, 'uncollectable': 0}, {'collections': 17, 'collected': 937, 'uncollectable': 0}]


In [ ]:
## Example code to push gen 0/1/2

import gc

def make_cycles(n):
    objs = []
    for _ in range(n):
        a = []
        b = []
        a.append(b)
        b.append(a)   # cycle a <-> b
        objs.append(a)
        objs.append(b)
    # drop external references; only cycles remain
    del objs

gc.collect()
print("Before:", gc.get_stats())

make_cycles(10000)

gc.collect()
print("After:", gc.get_stats())


Before: [{'collections': 274, 'collected': 1445, 'uncollectable': 0}, {'collections': 24, 'collected': 909, 'uncollectable': 0}, {'collections': 18, 'collected': 937, 'uncollectable': 0}]
After: [{'collections': 300, 'collected': 1445, 'uncollectable': 0}, {'collections': 26, 'collected': 909, 'uncollectable': 0}, {'collections': 19, 'collected': 20937, 'uncollectable': 0}]


#### Memory Management Best Practices

1. Use local variables: wherever possible instead of global variables as they have shorter life span and can be freed sooner

2. Avoid Circular references: Circular references will result in memory leak if not managed properly

3. Use Generators: Generators produce items one at a time and keep only one in the memory at a time, making memory more efficient

4. Explicitly delete objects: Use the del statement to explicitly delete the objects and variables

5. Profile Memory Usage: Use profiling tools like tracemalloc and memory profiler to identify memory leaks and opitmize memory usage



In [31]:
## gc.collect() practical usage

import gc
import sys

## create a class
class MyObject:
    ## constructor to initiate the class
    def __init__(self, name):
        self.name=name
        print(f'Object {self.name} created')

    ## to indicate we can access the objects without external references after deleting them
    def __del__(self):
        print(f'Object {self.name} deleted')

## create objects
obj1 = MyObject("obj1")
obj2 = MyObject("obj2")

## to form a reference cycle i.e., a<->b
obj1.ref = obj2
obj2.ref = obj1

print(f'Reference Count of obj1 is {sys.getrefcount(obj1)}')
print(f'Reference Count of obj2 is {sys.getrefcount(obj2)}')

## delete the external objects
del obj1
del obj2

## now the external objects obj1 and obj2 got deleted, but the reference cycle was not deleted and we can't access that object as there are no external references to point that object
## from the above refcount, we have created 3 objects, but using del we have deleted only 2 objects and can't access the 3rd one

## To solve this

Object obj1 created
Object obj2 created
Reference Count of obj1 is 3
Reference Count of obj2 is 3


In [ ]:
## Handled Circular References

## gc.collect() practical usage

import gc
import sys

## create a class
class MyObject:
    ## constructor to initiate the class
    def __init__(self, name):
        self.name=name
        print(f'Object {self.name} created')

    ## to indicate we can access the objects without external references after deleting them
    def __del__(self):
        print(f'Object {self.name} deleted')

## create objects
obj1 = MyObject("obj1")
obj2 = MyObject("obj2")

## to form a reference cycle i.e., a<->b
obj1.ref = obj2
obj2.ref = obj1

print(f'Reference Count of obj1 is {sys.getrefcount(obj1)}')
print(f'Reference Count of obj2 is {sys.getrefcount(obj2)}')

## delete the external objects
del obj1
del obj2

## now the external objects obj1 and obj2 got deleted, but the reference cycle was not deleted and we can't access that object as there are no external references to point that object
## from the above refcount, we have created 3 objects, but using del we have deleted only 2 objects and can't access the 3rd one

## to delete the 3rd object, reference cycle
gc.collect()

## last number in the output indicates total number of objects memory got de-allocated using gc.collect()

Object obj1 created
Object obj2 created
Reference Count of obj1 is 3
Reference Count of obj2 is 3
Object obj1 deleted
Object obj2 deleted
Object obj1 deleted
Object obj2 deleted


19

In [ ]:
## Generators for memory efficiency

def generate_number(num):
    for i in range(num):
        ## returns the immediate value and exits the function
        return i
    
for num in generate_number(1000):
    ## can't iterate over single int value
    print(num)
    if num>10:
        break

## To solve this

TypeError: 'int' object is not iterable

In [40]:
def generate_number(num):
    for i in range(num):
        ## returns the generator object
        yield i
    
for num in generate_number(1000):
    ## iterable over the generator object
    print(num)
    if num>10:
        break

## Generators produce only one value at a time and stores only one value at a time
## This will help us in saving the memory

0
1
2
3
4
5
6
7
8
9
10
11


In [49]:
## Profiling the memory using tracemalloc

import tracemalloc

## create a list of 1000 integers
def create_list():
    return [i for i in range(1000)]

def main():

    tracemalloc.start()                         ## begins tracking all memory allocations done by Python after this call, where and how much allocated

    create_list()

    snapshot = tracemalloc.take_snapshot()      ## take the snapshot of memory allocation
    top_stats= snapshot.statistics('lineno')    ## to return stats of the objects like size, number of blocks of memory and where

    print('Top 10 Stats: ')                     
    
    for stat in top_stats[:10]:                 ## top 10 objects based on size in descending order
        print(stat)

main()                                          ## call main

Top 10 Stats: 
/opt/anaconda3/lib/python3.12/site-packages/IPython/core/compilerop.py:174: size=38.0 KiB, count=374, average=104 B
<frozen genericpath>:89: size=36.0 KiB, count=309, average=119 B
/opt/anaconda3/lib/python3.12/tracemalloc.py:558: size=22.4 KiB, count=439, average=52 B
/opt/anaconda3/lib/python3.12/site-packages/pygments/style.py:94: size=18.1 KiB, count=287, average=65 B
/opt/anaconda3/lib/python3.12/site-packages/pygments/lexer.py:488: size=17.2 KiB, count=220, average=80 B
/opt/anaconda3/lib/python3.12/site-packages/IPython/core/compilerop.py:86: size=17.1 KiB, count=198, average=89 B
/opt/anaconda3/lib/python3.12/ast.py:52: size=15.5 KiB, count=174, average=91 B
/opt/anaconda3/lib/python3.12/site-packages/pygments/formatters/terminal256.py:44: size=14.4 KiB, count=290, average=51 B
/opt/anaconda3/lib/python3.12/tokenize.py:576: size=12.1 KiB, count=224, average=55 B
<string>:1: size=11.9 KiB, count=95, average=128 B
